*SHENASA FACE ANTI-SPOOFING*

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd gdrive/MyDrive/shenasa

Mounted at /content/gdrive


In [ ]:
import cv2
import tensorflow as tf
from keras.models import model_from_yaml
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array
from rPPG.rPPG_Extracter import *
from rPPG.rPPG_lukas_Extracter import *
import os
import shutil
import keras
import time
import numpy as np
from shutil import copyfileme
from progressbar import ProgressBar
from tensorflow.keras.callbacks import EarlyStopping
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
def frames_(video_name):
  """
  this function gets all the frames of a video (used for train dataset)
  the output frames will be in train/1/image/{name}


  parameters:
    video_name: name of the video that will be processed 

  return:
    None

  """
  # Opens the Video file
  name = video_name.split('.')[0]
  cap= cv2.VideoCapture(f'train/1/{video_name}')
  if not os.path.isdir(f"train/1/image/{name}"):
    os.mkdir(f"train/1/image/{name}")
  i=0
  # print(cap.isOpened())
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      cv2.imwrite(f'train/1/image/{name}/'+str(i)+'.jpg',frame)
      i+=1

In [ ]:
def frames_D(video_name):
  """
  this function gets all the frames of a video (used for test dataset)
  the output frames will be in test/1/image/{name}

  parameters:
    video_name: name of the video that will be processed 

  return:
    None

  """
  # Opens the Video file
  name = video_name.split('.')[0]
  cap= cv2.VideoCapture(f'test/1/{video_name}')
  if not os.path.isdir(f"test/1/image/{name}"):
    os.mkdir(f"test/1/image/{name}")
  i=0
  # print(cap.isOpened())
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      cv2.imwrite(f'test/1/image/{name}/'+str(i)+'.jpg',frame)
      i+=1

In [ ]:
def eval_frames(): 
  """
  this function select some frames from a frame file, add the face cordinations
   and name of the video as the name of the file and save it in test/frane for
   further processes

  parameters:
    None 

  return:
    None
  """ 
  ma_films = os.listdir(f'test/1')
  for j, i in enumerate(ma_films):
    if 'avi' in i:

      name = i.split('.')[0]
      print(name)

      ma_text = name + '.txt'

      with open('test/1/'+ma_text) as f:
        lines = f.readlines()
      ma_face = [line[:-1] for line in lines]

      frames_D(i)

      ma_len = len(os.listdir(f"test/1/image/{name}/"))
      
      
      for counter in range(0,ma_len,ma_len//5):
        face = list(map(int, ma_face[counter].split(',')))[1:]
        name1 = ','.join(map(str,face))
        copyfile(f"test/1/image/{name}/{counter}.jpg", f"test/frame/{name}+{counter}+{name1}.jpg")
      
      shutil.rmtree(f"test/1/image/{name}/")
      print(j)

In [ ]:
dim = (128,128)

In [ ]:
def get_rppg_pred(frame):
    """
    this function is used to return the rppg predictions of images

    parameters:
      the specific frame

    return:
      the rppg predictaion
    """
    use_classifier = True  
                                 
    sub_roi = []           
    use_resampling = False  
    
    fs = 20

    timestamps = []
    time_start = [0]

    break_ = False

    rPPG_extracter = rPPG_Extracter()
    rPPG_Lukas_Extracter()
    bpm = 0
    
    dt = time.time()-time_start[0]
    time_start[0] = time.time()
    if len(timestamps) == 0:
        timestamps.append(0)
    else:
        timestamps.append(timestamps[-1] + dt)
        

    rPPG_extracter.measure_rPPG(frame,use_classifier,sub_roi) 
    rPPG = np.transpose(rPPG_extracter.rPPG)
    
        # Extract Pulse
    if rPPG.shape[1] > 10:
        if use_resampling :
            t = np.arange(0,timestamps[-1],1/fs)
            
            rPPG_resampled= np.zeros((3,t.shape[0]))
            for col in [0,1,2]:
                rPPG_resampled[col] = np.interp(t,timestamps,rPPG[col])
            rPPG = rPPG_resampled
        num_frames = rPPG.shape[1]

    return rPPG

In [ ]:
def make_pred(li):
    """
    this function predict a single image recording to its rppg prediction and the image itself

    parameters:
      list: a list with first element the image and second the rppg prediction

    return 
      prediction that shows if the image is an attack or if it is not
    """
    [single_img,rppg] = li
    print(single_img.shape)
    single_img = cv2.resize(single_img, dim)
    single_x = img_to_array(single_img)
    single_x = np.expand_dims(single_x, axis=0)

    print(single_img.shape)
    print('============')
    print(single_x.shape)
    print('============')
    print(rppg.shape)

    single_pred = model.predict([single_x,rppg])
    return single_pred

In [ ]:
def images_pred(name):
  """
  this function predict frames of a specific video

  parameters:
    name: video name without .avi
  
  return:
    list of predictions
  """

  f_name = name+'.avi'
  ma_pred = []
  frames_(f_name)
  ma_face = np.loadtxt(f"train/1/{name}.txt", dtype=str)
  for counter, i in enumerate(os.listdir(f'train/1/image/{name}/')):
    im = cv2.imread(f"train/1/image/{name}/{i}")
    face = list(map(int, ma_face[counter].split(',')))[1:]

    x, y, w, h = face
    sub_img=im[y:y+h,x:x+w]
    
    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T
    pred = make_pred([sub_img,rppg_s])
    ma_pred.append(pred)
  return ma_pred

In [ ]:
def images_evaluate(model, path):
  """
  this function is used to evaluate some images in a specific directory

  parameters:
    model: the model that is used to predict images
    path: the directory in which the images are

  return:
    evaluation scores of the images
  """
  ma_frames = os.listdir(f'dev/{path}')

  list_labels = []
  ma_sub_imgs = []
  ma_rppg_ss = []

  for index, i in enumerate(ma_frames):
    label, counter, faces = i.split('+')
    faces = faces.split('.')[0]
    
    faces = list(map(int, faces.split(',')))
    im = cv2.imread(f"dev/{path}/{i}")

    label = label[-1]=='1'
    if label:
      list_labels.append([1,0])
    else:
      list_labels.append([0,1])

    x,y,w,h = faces
    sub_img = im[y:y+h,x:x+w]
    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T

    sub_img = cv2.resize(sub_img, dim)
    sub_img = img_to_array(sub_img)
    sub_img = np.expand_dims(sub_img, axis=0)

    sub_img = sub_img.reshape(128,128,3)
    rppg_s = rppg_s.reshape(3)

    ma_sub_imgs.append(sub_img)
    ma_rppg_ss.append(rppg_s)

    print(index)

  return model.evaluate(x=[np.array(ma_sub_imgs), np.array(ma_rppg_ss)], y=np.array(list_labels))

In [ ]:
# this script is used to delete files and folders from the trash of google drive

def login():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  my_drive = GoogleDrive(gauth)
  return my_drive
login()

# delete folder
def delte_from_trash():
  my_drive = login()
  while True:
    try:
      path = my_drive.ListFile({'q': "trashed = true"}).GetList()
      if len(path) == 0 : break
      for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
        a_file.Delete()
    except:
      my_drive = login()
delte_from_trash()

In [ ]:
def get_generator():
  """
  this is the main generator that is used to train the model
  the files in train/1 are used to train the model

  in each step it yields frames of a video
  """

  while True:
    ma_films = os.listdir(f'train/1')
    count = 0
    for i in ma_films:
      
      if count == 300:
        break

      if 'avi' in i:
       
        name = i.split('.')[0]

        # change this line if you want to use more or less frames
        number_frame = 30

        # this part can be modified according to your segmentation of real and
        # attack images... 
        # if you want to use every single file, delete these 2 lines
        if name[-1] in ['3','5']:
          continue

        print('filme:',name)

        count+=1
        print('   counter: ',count)

        ma_text = name + '.txt'
        with open('train/1/'+ma_text) as f:
          lines = f.readlines()
        ma_face = [line[:-1] for line in lines]

        frames_(i)
        
        list_images_array = []
        list_face_array = []

        ma_len = len(os.listdir(f"train/1/image/{name}/"))

        for counter in range(0,ma_len,ma_len//number_frame):

          im = cv2.imread(f"train/1/image/{name}/{counter}"+'.jpg')
          face = list(map(int, ma_face[counter].split(',')))[1:]
          flag = True
          if face == [0,0,0,0]:
            flag = False
            count -= 1
            break

          list_images_array.append(im)
          list_face_array.append(face)

        if flag == False:
          continue
          

        ma_sub_imgs = []
        ma_rppg_ss = []

        ma_sub_imgs = []
        ma_rppg_ss = []
        for j in range(len(list_images_array)):
          try:

            x, y, w, h = list_face_array[j]
            sub_img = list_images_array[j][y:y+h,x:x+w]

            rppg_s = get_rppg_pred(sub_img)
            rppg_s = rppg_s.T


            sub_img = cv2.resize(sub_img, dim)
            sub_img = img_to_array(sub_img)
            sub_img = np.expand_dims(sub_img, axis=0)

            sub_img = sub_img.reshape(128,128,3)
            rppg_s = rppg_s.reshape(3)

            ma_sub_imgs.append(sub_img)

            ma_rppg_ss.append(rppg_s)
          except:
            print()
            print(f"could not procces image number {j}")
            print()

        labels = []
        if '1' in name[-1]:
          labels.append([1,0])
          labels*=len(list_images_array)
        else:
          labels.append([0,1])
          labels*=len(list_images_array)
        print()
        print(f'  deleting file {name}')
        shutil.rmtree(f"train/1/image/{name}/")

        delte_from_trash()

        yield ([np.array(ma_sub_imgs), np.array(ma_rppg_ss)], np.array(labels))

In [ ]:
ma_generator = get_generator()

In [ ]:
def prediction_test():
  """
  this function simply returns prediction of frames in the test/frame directory
  """
  result = []
  ma_frames = os.listdir(f'test/frame')
  counter = 0
  for i in ma_frames:
    print('*'*20)
    print(counter)
    print('*'*20)
    im = cv2.imread(f'test/frame/{i}')
    face = list(map(int, i.split('+')[2].split('.')[0].split(',')))

    x, y, w, h = face
    sub_img=im[y:y+h,x:x+w]
    
    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T
    pred = make_pred([sub_img,rppg_s])
    result.append(pred)
    counter += 1
  result = np.array(result).reshape(3600, -1) 
  return tf.math.argmin(result, axis=1)

In [ ]:
def labels_test():
  """
  this function simply returns labels of frames in test/frame directory
  """
  result = []
  ma_frames = os.listdir(f'test/frame')
  for i in ma_frames:
    result.append(i.split('+')[0][-1])
  return result

In [ ]:
# this script is used to check out the confusion matrix 
p1 = prediction_test()
l1 = labels_test()
res = tf.math.confusion_matrix(tf.convert_to_tensor(list(map(int,l1))), p1)
print(res)

In [ ]:
# loading the model
json_file = open('RGB_rPPG_merge_softmax_.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("test_30_180_1.h5")

In [ ]:
# compiling the model :D
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
history = model.fit_generator(ma_generator, steps_per_epoch=180, epochs=1, verbose=1, callbacks=[ma_callbask])
# frame - film number - trained epoch 
model.save_weights("test_30_180_3.h5")

In [ ]:
def evaluate_score(model, path):
  # this function just evaluate and prints the evaluation scores of images in specific path.
  score = images_evaluate(model, path)
  print(path)
  print('Test loss:', score[0]) 
  print('Test accuracy:', score[1])

In [ ]:
evaluate_score(model, 'real')

In [ ]:
evaluate_score(model, 'replay')

In [ ]:
evaluate_score(model, 'print')

In [ ]:
evaluate_score(model, 'frame')